In [10]:
import os
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import joblib

# --- Configuration ---
# As per requirement 2, an 8x8 grid on an 800x600 image.
GRID_W = 100  # 800 / 8
GRID_H = 75   # 600 / 8
TARGET_ASPECT_RATIO = 4.0 / 3.0

# LBP parameters for texture analysis
LBP_RADIUS = 1
LBP_POINTS = 8 * LBP_RADIUS

# --- Image Preprocessing (Requirement 1) ---
def process_image_for_training(path: str) -> np.ndarray | None:
    """
    Loads an image and processes it according to the strict 4:3 / 800x600 rules.
    - Enforces 4:3 aspect ratio by center-cropping.
    - Scales down large images.
    - Discards small images.
    """
    img = cv2.imread(path)
    if img is None:
        print(f"Warning: Could not read image at {path}")
        return None
    
    h, w, _ = img.shape
    current_aspect_ratio = w / h
    
    # a. Crop to 4:3 aspect ratio if it's not already correct
    if not np.isclose(current_aspect_ratio, TARGET_ASPECT_RATIO):
        if current_aspect_ratio > TARGET_ASPECT_RATIO: # Image is wider than 4:3
            new_w = int(TARGET_ASPECT_RATIO * h)
            x_start = (w - new_w) // 2
            img = img[:, x_start:x_start + new_w]
        else: # Image is taller than 4:3
            new_h = int(w / TARGET_ASPECT_RATIO)
            y_start = (h - new_h) // 2
            img = img[y_start:y_start + new_h, :]
    
    # b. Scale down if larger than 800x600
    h, w, _ = img.shape
    if w > 800 or h > 600:
        img = cv2.resize(img, (800, 600), interpolation=cv2.INTER_AREA)

    # c. Do not scale up if smaller than 800x600 (discard)
    h, w, _ = img.shape
    if w < 800 or h < 600:
        # print(f"Info: Discarding small image: {path} ({w}x{h})")
        return None
        
    # Convert to grayscale for feature extraction
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# --- Feature Extraction (Same as before, but on processed images) ---
def extract_features_from_cell(cell: np.ndarray) -> np.ndarray:
    """Extracts features from a single 100x75 grid cell."""
    features = []
    features.append(np.mean(cell))
    features.append(np.std(cell))
    lbp = local_binary_pattern(cell, LBP_POINTS, LBP_RADIUS, method="uniform")
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, LBP_POINTS + 3), density=True)
    features.extend(hist)
    edges = cv2.Canny(cell, 50, 150)
    features.append(np.sum(edges > 0) / cell.size)
    return np.array(features)

def extract_features_from_image(img: np.ndarray) -> np.ndarray:
    """Extracts features from all 64 cells in a single image."""
    features_list = []
    for y in range(0, img.shape[0], GRID_H):
        for x in range(0, img.shape[1], GRID_W):
            cell = img[y:y+GRID_H, x:x+GRID_W]
            features_list.append(extract_features_from_cell(cell))
    return np.vstack(features_list)

# --- Main Training Pipeline ---
if __name__ == "__main__":
    training_folder = "images"
    model_output_path = "wildlife_detector_model.pkl"
    
    os.makedirs(training_folder, exist_ok=True)

    print("--- Starting Model Training ---")
    image_paths = [os.path.join(training_folder, f) for f in os.listdir(training_folder) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
    
    if not image_paths:
        print(f"Error: No images found in '{training_folder}'. Please add images to train the model.")
    else:
        all_features = []
        print(f"Processing {len(image_paths)} images from '{training_folder}'...")
        for path in image_paths:
            processed_img = process_image_for_training(path)
            if processed_img is not None:
                features = extract_features_from_image(processed_img)
                all_features.append(features)

        if not all_features:
            print("Error: No valid images found for training after processing. Check image dimensions.")
        else:
            X_train = np.vstack(all_features)
            print(f"Successfully extracted {X_train.shape[0]} feature vectors for training.")
            
            # 1. Create and fit the scaler
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            
            # 2. Create and fit the Isolation Forest model
            # Contamination is the expected % of cells containing wildlife. Adjust if needed.
            iso_forest = IsolationForest(contamination=0.1, random_state=42, n_jobs=-1)
            iso_forest.fit(X_train_scaled)
            
            # 3. Save both models to a single pickle file
            models = {'scaler': scaler, 'iso_forest': iso_forest}
            joblib.dump(models, model_output_path)
            
            print("\n--- Training Complete ---")
            print(f"Scaler and Isolation Forest models have been saved to '{model_output_path}'")


--- Starting Model Training ---
Processing 465 images from 'images'...
Successfully extracted 27584 feature vectors for training.

--- Training Complete ---
Scaler and Isolation Forest models have been saved to 'wildlife_detector_model.pkl'


In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
import joblib
import random

# --- Configuration (Must match the model's training script) ---
GRID_W = 100
GRID_H = 75
TARGET_ASPECT_RATIO = 4.0 / 3.0
LBP_RADIUS = 1
LBP_POINTS = 8 * LBP_RADIUS
MODEL_PATH = "wildlife_detector_model.pkl"
IMAGE_FOLDER = "images"

# --- Helper Functions (Copied from generate_output.py) ---

def process_image_for_prediction(path: str) -> np.ndarray | None:
    """Loads and processes an image using the same rules as training."""
    img = cv2.imread(path)
    if img is None: return None
    h, w, _ = img.shape
    current_aspect_ratio = w / h
    if not np.isclose(current_aspect_ratio, TARGET_ASPECT_RATIO):
        if current_aspect_ratio > TARGET_ASPECT_RATIO:
            new_w = int(TARGET_ASPECT_RATIO * h)
            x_start = (w - new_w) // 2
            img = img[:, x_start:x_start + new_w]
        else:
            new_h = int(w / TARGET_ASPECT_RATIO)
            y_start = (h - new_h) // 2
            img = img[y_start:y_start + new_h, :]
    h, w, _ = img.shape
    if w > 800 or h > 600:
        img = cv2.resize(img, (800, 600), interpolation=cv2.INTER_AREA)
    h, w, _ = img.shape
    if w < 800 or h < 600: return None
    return img

def extract_features_from_image(img: np.ndarray) -> np.ndarray:
    """
    Extracts features from all 64 cells in a single image.
    img: Input image (assumed to be in BGR format).
    Returns a 64xN feature array, where N is the number of features per cell
    """
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features_list = []
    for y in range(0, gray_img.shape[0], GRID_H):
        for x in range(0, gray_img.shape[1], GRID_W):
            cell = gray_img[y:y+GRID_H, x:x+GRID_W]
            # --- Feature Extraction Logic (must be identical to training) ---
            cell_features = []
            cell_features.append(np.mean(cell))
            cell_features.append(np.std(cell))
            lbp = local_binary_pattern(cell, LBP_POINTS, LBP_RADIUS, method="uniform")
            hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, LBP_POINTS + 3), density=True)
            cell_features.extend(hist)
            edges = cv2.Canny(cell, 50, 150)
            cell_features.append(np.sum(edges > 0) / cell.size)
            features_list.append(np.array(cell_features))
    return np.vstack(features_list)

def draw_grid_visualization(img: np.ndarray, grid_map: np.ndarray) -> np.ndarray:
    """Draws the final visualization."""
    def apply_dither_effect(cell: np.ndarray) -> np.ndarray:
        h, w, _ = cell.shape
        overlay = np.zeros_like(cell, dtype=np.uint8)
        for y in range(0, h, 4):
            for x in range(0, w, 4):
                cv2.circle(overlay, (x, y), 1, (200, 200, 200), -1)
        return cv2.addWeighted(cell, 0.5, overlay, 0.5, 0)

    cell_number = 1
    for i in range(8): # 8 rows
        for j in range(8): # 8 cols
            y1, x1 = i * GRID_H, j * GRID_W
            y2, x2 = y1 + GRID_H, x1 + GRID_W
            if grid_map[i, j] == 1:
                img[y1:y2, x1:x2] = apply_dither_effect(img[y1:y2, x1:x2])
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 255), 1)
            cv2.putText(img, str(cell_number), (x1 + 3, y1 + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 255), 1)
            cell_number += 1
    return img

# --- Main Test Execution ---
if __name__ == "__main__":
    # 1. Check for model and image folder
    if not os.path.exists(MODEL_PATH):
        print(f"Error: Model file not found at '{MODEL_PATH}'. Please run train_model.py first.")
    elif not os.path.isdir(IMAGE_FOLDER):
        print(f"Error: Folder '{IMAGE_FOLDER}' not found. Please create it and add images.")
    else:
        # 2. Find a random image
        all_images = [f for f in os.listdir(IMAGE_FOLDER) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
        if not all_images:
            print(f"No images found in '{IMAGE_FOLDER}'.")
        else:
            random_image_name = random.choice(all_images)
            image_path = os.path.join(IMAGE_FOLDER, random_image_name)
            print(f"--- Testing random image: {random_image_name} ---")

            # 3. Load models
            models = joblib.load(MODEL_PATH)
            scaler = models['scaler']
            iso_forest = models['iso_forest']

            # 4. Process the image
            processed_img = process_image_for_prediction(image_path)
            if processed_img is None:
                print("Image was discarded as it did not meet the size/aspect ratio requirements.")
            else:
                # 5. Extract features, scale, and predict
                features = extract_features_from_image(processed_img)
                features_scaled = scaler.transform(features)
                preds = iso_forest.predict(features_scaled)

                # 6. Visualize and show the result
                grid_map = (preds == -1).astype(int).reshape((8, 8))
                final_image = draw_grid_visualization(processed_img.copy(), grid_map)

                # Save the output for review
                output_filename = f"test_output_{random_image_name}"
                cv2.imwrite(output_filename, final_image)
                print(f"Output saved to '{output_filename}'")
                
                # Display in a window
                cv2.imshow(f"Test Result: {random_image_name}", final_image)
                print("Press any key to close the image window.")
                cv2.waitKey(0)
                cv2.destroyAllWindows()

--- Testing random image: CIMG0135.JPG ---
Output saved to 'test_output_CIMG0135.JPG'
Press any key to close the image window.
